In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=150,
	memory_key="chat",
	return_messages=True
)

examples = [
	{
		"movie_name": "대부",
		"answer": "👨‍👨‍👦🔫🍝"
	},
	{
		"movie_name": "탑건",
		"answer": "🛩️👨‍✈️🔥"
	},
	{
		"movie_name": "캡틴아메리카: 더 윈터 솔져",
		"answer": "🇺🇸❄️💥",
	},
	{
		"movie_name": "슬램덩크",
		"answer": "🏀🔥🤯",
	}
]

example_prompt = ChatPromptTemplate.from_messages([
	("human", "{movie_name}"),
	("ai", "{answer}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
)

prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 훌륭한 영화 평론가입니다. 영화 제목을 들으면 단 세개의 이모티콘으로 영화를 표현할 수 있습니다."),
	MessagesPlaceholder(variable_name="chat"),
	few_shot_prompt,
	("human", "{movie_name}"),
])

def load_memory(_):
	return memory.load_memory_variables({})["chat"]

chain = RunnablePassthrough.assign(chat=load_memory) | prompt | llm

def invoke_chain(movie_name):
	result = chain.invoke({"movie_name": movie_name})
	memory.save_context(
		{"input": movie_name},
		{"output": result.content},
	)
	print(result.content)

In [67]:
invoke_chain("엘리멘탈")

🔥💧🌪️


In [68]:
invoke_chain("인사이드 아웃2")

🧠💥😂


In [69]:
memory.load_memory_variables({})["chat"]

[HumanMessage(content='엘리멘탈'),
 AIMessage(content='🔥💧🌪️'),
 HumanMessage(content='인사이드 아웃2'),
 AIMessage(content='🧠💥😂')]

In [70]:
invoke_chain("내가 처음 질문한 영화의 제목은 무엇이지?")

처음 질문하신 영화는 "엘리멘탈"입니다.
